# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [32]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

In [33]:
[program['id'] for program in data['programs']]

['qasm3-runner', 'sampler', 'estimator', 'circuit-runner']

## Get temporary Access token from Auth API via API Token

In [34]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [35]:
url_backends = 'https://runtime-us-east.quantum-computing.ibm.com/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

In [36]:
print(backends_response.json()['devices'][:5],"...")

['simulator_stabilizer', 'simulator_extended_stabilizer', 'simulator_statevector', 'simulator_mps', 'simulator_stabilizer'] ...


## Run a job

### set up qasm circuit

In [37]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [38]:
import requests

url = 'https://runtime-us-east.quantum-computing.ibm.com/jobs'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}
job_input = {
    'program_id': 'sampler',
    "backend": 'ibm_wellington',
    "hub": "ibm-q-internal",
    "group": "dev-sys-software",
    "project": "internal-test",
    "params": {
        "circuits": [qasm_string]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cjvks5fp8blo0kgog530","backend":"ibm_wellington"}


## Wait for results

In [39]:
status_result = requests.get(url+'/'+job_id,headers=headers)
print(status_result.json()['state'],job_id)

{'status': 'Running'} cjvks5fp8blo0kgog530


### Get results

In [45]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': 0.0010648129545545889, '1': 0.9989351870454455}],
 'metadata': [{'shots': 4000,
   'readout_mitigation_overhead': 1.009828346656723,
   'readout_mitigation_time': 0.18535281000367831}]}

## Invalidate Token

In [41]:
#now trying to logout
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 
